# Basic SQL queries
This notebook is based on the Intro to SQL tutorial by Kaggle. 

In [1]:
import sqlite3

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.40.1


In [2]:
# Open or create database file
db = sqlite3.connect('./data/mydata_03.db')

# Create a cursor to execute SQL statements
cursor = db.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS pet_records(id INTEGER PRIMARY KEY, 
                   name TEXT, animal TEXT, date TEXT)
''')
db.commit()

In [3]:
# Insert the data
cursor.execute('''INSERT INTO pet_records values (1, 'Francisco Perez', 'Gato', '2023-12-23')''')
cursor.execute('''INSERT INTO pet_records values (2, 'Diego Abarca', 'Gato', '2022-01-13')''')
cursor.execute('''INSERT INTO pet_records values (3, 'Jorge Barra', 'Perro', '2024-01-07')''')
cursor.execute('''INSERT INTO pet_records values (4, 'Carlos Rodriguez', 'Conejo', '2023-08-18')''')

db.commit()


## Group By, Having & Count
### Aggregate functions
COUNT() returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.

Other examples of aggregate functions include SUM(), AVG(), MIN(), and MAX().

In [4]:
import pandas as pd

query = '''
        SELECT COUNT(id)
        FROM pet_records
        '''

query_df = pd.read_sql_query(query, db)
query_df.head()

,COUNT(id)
0,4


### GROUP BY
GROUP BY takes the name of one or more columns, and treats all rows with the same value in that column as a single group when you apply aggregate functions like COUNT().



In [5]:
query = '''
        SELECT animal, COUNT(id)
        FROM pet_records
        GROUP BY animal
        '''

query_df = pd.read_sql_query(query, db)
query_df.head()

,animal,COUNT(id)
0,Conejo,1
1,Gato,2
2,Perro,1


### GROUP BY ... HAVING
HAVING is used in combination with GROUP BY to ignore groups that don't meet certain criteria.

Also, you can change the name by adding AS after you specify the aggregation

In [6]:
query = '''
        SELECT animal, COUNT(id) AS num_animals
        FROM pet_records
        GROUP BY animal
        HAVING COUNT(id) > 1
        '''

query_df = pd.read_sql_query(query, db)
query_df.head()

,animal,num_animals
0,Gato,2


### WHERE ... LIKE
A WHERE clause can limit your results to rows with certain text using the LIKE feature.

You can also use `%` as a "wildcard" for any number of characters.

In [7]:
# You can filter by an specific text
query = '''
        SELECT name, animal
        FROM pet_records
        WHERE animal LIKE 'Perro'
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())
print('\n')

# or use the % as a wildcard. In this case the word 'at' will also be in the result
query = '''
        SELECT name, animal
        FROM pet_records
        WHERE animal LIKE '%at%'
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())

          name animal
0  Jorge Barra  Perro


              name animal
0  Francisco Perez   Gato
1     Diego Abarca   Gato


## Order by and Dates
### ORDER BY
ORDER BY is usually the last clause in your query, and it sorts the results returned by the rest of your query.

In [8]:
# By default order is ascending
query = '''
        SELECT id, name, animal
        FROM pet_records
        ORDER BY id
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())
print('\n')

# It also work on columns containing text
query = '''
        SELECT id, name, animal
        FROM pet_records
        ORDER BY name
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())
print('\n')

# You can reverse the order using the DESC argument
query = '''
        SELECT id, name, animal
        FROM pet_records
        ORDER BY name DESC
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())

   id              name  animal
0   1   Francisco Perez    Gato
1   2      Diego Abarca    Gato
2   3       Jorge Barra   Perro
3   4  Carlos Rodriguez  Conejo


   id              name  animal
0   4  Carlos Rodriguez  Conejo
1   2      Diego Abarca    Gato
2   1   Francisco Perez    Gato
3   3       Jorge Barra   Perro


   id              name  animal
0   3       Jorge Barra   Perro
1   1   Francisco Perez    Gato
2   2      Diego Abarca    Gato
3   4  Carlos Rodriguez  Conejo


## Dates
### Extract day, month and year
SQLite does not have a storage class set aside for storing dates and/or times. Instead, the built-in Date And Time Functions of SQLite are capable of storing dates and times as TEXT, REAL, or INTEGER values.

In TEXT, the format follows the ISO8601 strings ("YYYY-MM-DD HH:MM:SS.SSS").

In SQLite, we can use the strftime() function to return datetime values in our chosen format:

%d &emsp; day of month: 00-31

%e &emsp; day of month without leading zero: 0-31

%f &emsp; fractional seconds: SS.SSS

%F &emsp; ISO 8601 date: YYYY-MM-DD

%H &emsp; hour: 00-24

%I &emsp; hour for 12-hour clock: 01-12

%j &emsp; day of year: 001-366

%J &emsp; Julian day number (fractional)

%k &emsp; hour without leading zero: 0-24

%l &emsp; %I without leading zero: 1-12

%m &emsp; month: 01-12

%M &emsp; minute: 00-59

%p &emsp; "AM" or "PM" depending on the hour

%P &emsp; "am" or "pm" depending on the hour

%R &emsp; ISO 8601 time: HH:MM

%s &emsp; seconds since 1970-01-01

%S &emsp; seconds: 00-59

%T &emsp; ISO 8601 time: HH:MM:SS

%u &emsp; day of week 1-7 with Monday==1

%w &emsp; day of week 0-6 with Sunday==0

%W &emsp; week of year: 00-53

%Y &emsp; year: 0000-9999

In [9]:
query = '''
        SELECT name,
            strftime('%d', date) AS "day",
            strftime('%m', date) AS "month",
            strftime('%Y', date) AS "year"
        FROM pet_records
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())
print('\n')

# Extract the day of the week using %u. It outputs a number from 1-7 with Monday==1
query = '''
        SELECT name,
            strftime('%w', date) AS "weekday"
        FROM pet_records
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())


               name day month  year
0   Francisco Perez  23    12  2023
1      Diego Abarca  13    01  2022
2       Jorge Barra  07    01  2024
3  Carlos Rodriguez  18    08  2023


               name weekday
0   Francisco Perez       6
1      Diego Abarca       4
2       Jorge Barra       0
3  Carlos Rodriguez       5


## As & With
Organize your query for better readability. This becomes especially important for complex queries.

### WITH ... AS
A common table expression (or CTE) is a temporary table that you return within your query. CTEs are helpful for splitting your queries into readable chunks, and you can write queries against them.

In [10]:
# Get the owner name of young dogs (born on 2023+)
query = '''
        WITH young_pets AS
        (
            SELECT id, name, animal
            FROM pet_records
            WHERE date > '2023-01-01'
        )
        SELECT name
        FROM young_pets
        WHERE animal = 'Perro'
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())


          name
0  Jorge Barra


## Joining Data
### JOIN
Using JOIN, we can write a query to create a table, combining information from both tables by matching rows.

The type of JOIN we're using is called an INNER JOIN. That means that a row will only be put in the final output table if the value in the columns where in both the tables you're joining.

In [11]:
cursor.execute('''CREATE TABLE IF NOT EXISTS owners(id INTEGER PRIMARY KEY, 
                   name TEXT, pet_id INTEGER)
''')
cursor.execute('''INSERT INTO owners values (1, 'Francisco Perez', 1)''')
cursor.execute('''INSERT INTO owners values (2, 'Diego Abarca', 2)''')
cursor.execute('''INSERT INTO owners values (3, 'Jorge Barra', 4)''')
cursor.execute('''INSERT INTO owners values (4, 'Carlos Rodriguez', 3)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS pets(id INTEGER PRIMARY KEY, 
                   name TEXT, animal TEXT)
''')
cursor.execute('''INSERT INTO pets values (1, 'Puppy', 'Gato')''')
cursor.execute('''INSERT INTO pets values (2, 'Washington', 'Gato')''')
cursor.execute('''INSERT INTO pets values (3, 'Laika', 'Conejo')''')
cursor.execute('''INSERT INTO pets values (4, 'Mapa', 'Perro')''')
db.commit()

In [12]:
query = '''
        SELECT *
        FROM owners
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())
print('\n')

query = '''
        SELECT *
        FROM pets
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())
print('\n')


query = '''
        SELECT o.name AS owner_name,
               p.name AS pet_name,
               p.animal AS animal
        FROM pets AS p
        INNER JOIN owners AS o
            ON p.id = o.pet_id
        '''

query_df = pd.read_sql_query(query, db)
print(query_df.head())
print('\n')

   id              name  pet_id
0   1   Francisco Perez       1
1   2      Diego Abarca       2
2   3       Jorge Barra       4
3   4  Carlos Rodriguez       3


   id        name  animal
0   1       Puppy    Gato
1   2  Washington    Gato
2   3       Laika  Conejo
3   4        Mapa   Perro


         owner_name    pet_name  animal
0   Francisco Perez       Puppy    Gato
1      Diego Abarca  Washington    Gato
2       Jorge Barra        Mapa   Perro
3  Carlos Rodriguez       Laika  Conejo




In [13]:
cursor.execute('''DROP TABLE pets''')
cursor.execute('''DROP TABLE owners''')
cursor.execute('''DROP TABLE pet_records''')
db.commit()